In [1]:
import cv2
import numpy as np
from PIL import Image, ImageOps, ImageDraw
from scipy.ndimage import morphology, label
from keras import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
import os
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import model_from_json
import sys
import numpy
from PIL import Image, ImageOps, ImageDraw
from scipy.ndimage import morphology, label

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = 'IMG-3077.jpg' #can modify
img1234 = cv2.imread(path)
cv2_im = cv2.cvtColor(img1234,cv2.COLOR_BGR2RGB)
pil_im = Image.fromarray(cv2_im)

In [3]:
def boxes(orig):
    img = ImageOps.grayscale(orig)
    im = numpy.array(img)

    # Inner morphological gradient.
    im = morphology.grey_dilation(im, (3, 3)) - im

    # Binarize.
    mean, std = im.mean(), im.std()
    t = mean + std
    im[im < t] = 0
    im[im >= t] = 1

    # Connected components.
    lbl, numcc = label(im)
    # Size threshold.
    min_size = 200 # pixels
    box = []
    print("Number of connected components is: " + str(numcc))
    for i in range(1, numcc + 1):
        py, px = numpy.nonzero(lbl == i)
        if len(py) < min_size:
            im[lbl == i] = 0
            continue

        xmin, xmax, ymin, ymax = px.min(), px.max(), py.min(), py.max()
        # Four corners and centroid.
        box.append([
            [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)],
            (numpy.mean(px), numpy.mean(py))])

    print("finished running boxes!")
    return im.astype(numpy.uint8) * 255, box

orig = pil_im
im, box = boxes(orig)


# Boxes found.
def dist(t1, t2):
    return np.sqrt((t1[0] - t2[0])**2 + (t1[1] - t2[1])**2)
# Draw perfect rectangles and the component centroid.
img = Image.fromarray(im)
visual = img.convert('RGB')
draw = ImageDraw.Draw(visual)
box1 = []
counter = 1
for b,centroid in box:
    for b1, centroid1 in box[counter:]:
        if dist(b[0], b1[0]) < 50 and (abs((b[1][0] - b[0][0]) - (b1[1][0] - b1[0][0])) < 50) and (abs((b[2][1] - b[0][1]) - (b1[2][1] - b1[0][1])) < 50):
            if (b[1][0] - b[0][0]) < (b1[1][0] - b1[0][0]):
                smaller = b
                smallercent = centroid
            else:
                smaller = b1
                smallercent = centroid1
            box1.append([smaller, smallercent])
    counter+=1
print(box1)
for x in box1:
    try:
        box.remove(x)
    except:
        continue
print(box)
for b, centroid in box:
    draw.line(b + [b[0]], fill='yellow')
    cx, cy = centroid
    draw.ellipse((cx - 2, cy - 2, cx + 2, cy + 2), fill='red')
#visual.save(sys.argv[3])
opencvImage = cv2.cvtColor(numpy.array(visual), cv2.COLOR_RGB2BGR)
cv2.imwrite('testBoxes.png', opencvImage)

Number of connected components is: 12505
finished running boxes!
[[[(975, 2940), (1007, 2940), (1007, 2954), (975, 2954)], (991.6425531914894, 2947.391489361702)]]
[[[(70, 303), (3503, 303), (3503, 1640), (70, 1640)], (1760.3305423973757, 1147.422299195981)], [[(2569, 333), (2694, 333), (2694, 342), (2569, 342)], (2632.2286282306163, 337.7097415506958)], [[(109, 343), (3456, 343), (3456, 1612), (109, 1612)], (1688.766444032591, 971.4900688883449)], [[(384, 547), (1304, 547), (1304, 1424), (384, 1424)], (872.6809711540533, 940.0813460005601)], [[(2298, 558), (3142, 558), (3142, 1535), (2298, 1535)], (2689.6226055872867, 1016.3489917239772)], [[(656, 561), (1177, 561), (1177, 631), (656, 631)], (916.9293954776189, 600.9935394554684)], [[(2518, 584), (3071, 584), (3071, 608), (2518, 608)], (2770.1892690130444, 595.9778488801378)], [[(2369, 586), (2486, 586), (2486, 592), (2369, 592)], (2426.2525951557095, 589.3910034602076)], [[(127, 747), (141, 747), (141, 800), (127, 800)], (132.3852459

True

In [7]:
class Box():
    def __init__(self, img):
        self.img = img
        self.children = []
box.sort(reverse=True, key= lambda x: abs(x[0][0][0] - x[0][1][0]) * abs(x[0][0][1] - x[0][2][1]))

boxes = Box(img1234[box[0][0][0][1]: box[0][0][2][1], box[0][0][0][0]:box[0][0][1][0]])
counter1 = 1
for b,centroid in box[1:]:
    boxes.children.append(Box(img1234[b[0][1]: b[2][1], b[0][0]:b[1][0]]))
#cv2.imwrite('testBoxLarge.png', boxes.img)



In [8]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
from keras.preprocessing import image

In [9]:
from keras.preprocessing import image
def box_converter(box):
    if box == None:
        return
    if len(box.children) == 0:
        box.img = cv2.resize(box.img, (64, 64)) 
        test_image = image.img_to_array(box.img)
        test_image = np.expand_dims(test_image, axis = 0)
        result = loaded_model.predict(test_image)
        #training_set.class_indices
        if result[0][0] == 1:
            prediction = '<div/>'
        else:
            prediction = '<checkbox/>'
        return prediction
    elif len(box.children) > 0:
        header = '<div/>\n\t' 
        chil = [box_converter(i) for i in box.children]
        footer = '\n</div>'
        toReturn = ''
        for i in chil:
            toReturn+=i+'\n\t'
        return header + toReturn+footer

In [10]:
print(box_converter(boxes))

<div/>
	<div/>
	<checkbox/>
	<div/>
	<div/>
	<div/>
	
</div>


<img src="testBoxes.png"/>